Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

NAME = "Rey Stone"
COLLABORATORS = "Mara Backsen"

---

# CSCI 3155 Spring 2025 Assignment 4 : Semantics

This assignment asks you to write scala programs. 

**Restrictions** apply to each problem in terms of forbidden Scala features and API functions. Please read them carefully and ask for clarifications from the course staff over Piazza or during office hours if unsure.

Note: `???` indicates that there is a missing function or code fragment that needs to be filled in. In scala, 
it is also a macro that throws a `NotImplemented` exception. Make sure that you remove the `???` and replace it with the answer. 

Use the test cases provided to test them. You are also encouraged to write your own test cases to help debug your work. However, please delete any extra cells you may have created lest they break our autograder.

**Very Important:** Please run the cell that defines the functions `passed` and `testWithMessage` below whenever you restart the notebook.

In [1]:
// TEST HELPER

// FIRST RUN THIS CELL EVERY TIME YOU START THE NOTEBOOK
def passed(points: Int) = {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

def testWithMessage[T](v1: T, expected: T, testID: String) = { 
    println(s"Test $testID"); 
    println(s"\t Your code returned: $v1, Expected: $expected")
    assert (v1 == expected, s"Test $testID FAILED.")
    println("\t Passed!")
}

defined function passed
defined function testWithMessage

## Problem 1 (50 points)

In this assignment, we will implement regular expression 
matching using semantic rules. H/T to Prof. Matt Might : https://matt.might.net/articles/implementation-of-regular-expression-matching-in-scheme-with-derivatives/ 

$$\newcommand\Regex{\mathbf{Regex}}
\newcommand\trm[1]{\mathit{#1}}$$

The grammar of regular expressions is given by 
$$\begin{array}{rll}
\Regex & \Rightarrow \trm{Atom}(\mathbf{String}) & \text{match the string}\\ 
& | \ \trm{EmptyStr} & \text{match empty string} \\ 
& | \ \trm{Null} & \text{a regular expression that does not match any string} \\ 
& |\ \trm{Seq}(\Regex, \Regex) & \text{match the first regex and then the second}\\ 
& |\ \trm{Or}(\Regex, \Regex) & \text{Or of two regular expressions}\\ 
& |\ \trm{Star} (\Regex) & \text{Kleene-star: match zero or more occurrences of regex}\\ 
\end{array} $$ 

A regex such as `("hello")* | ( "world" ; "csci3155) *` is expressed in our abstract syntax by the term
~~~
Or( 
    Star(Atom("hello")), 
    Star( Seq( Atom("world"), Atom("csci3155")) 
  )
~~~

Note that `Null` and `EmptyStr` do not correspond to anything that a user writes. But we need them to implement the semantics. In general, it is possible to have abstract syntax that does not correspond to any concrete syntax. We will see an example of this when we implement the "Lettuce" language in class.

### Regular Expression Matching

Given a Regex `("hello")* | ( "world" ; "csci3155) *` 
 - The string `"worldcsci3155worldcsci3155"` matches the regex. 
 - Likewise the string `"hellohellohello"` also matches
the regex. 
 - However, the string `"helloworld"` does not match the regex.

We will use semantic rules to write a matcher function `match(str, regex)` that returns `true` if a string matches a regular expression and `false` otherwise. 

To do so, we proceed in three stages: 
  - We will define the `acceptsEmpty` function.
  - We will define the `simplifyNull` function to remove `null` wherever possible.
  - We will define the `derivative` of a regex with respect to a character.
  - We will finally put the two together to define a `match` function. 
  
Strap on your seatbelts and join us for the regex match!!

**Note** Below you'll see a lot of semantic rules. **Do not panic!** Semantic rules are simply describing the regex matching algorithm using Math instead of English. You do not have to write any new rules in this assignment. You simply have to translate the algorithm described as semantic rules into Scala code. Each rule generally corresponds to a `case` in the the `match-case` expression, so translating rules to code should be straightforward.




### The acceptsEmpty Function

The acceptsEmpty function described below uses the semantic rules to specify whether a regular expression accepts the _empty string_.


$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}\ (\text{#3}) }\newcommand\acc{\mathsf{acceptsEmpty}} \newcommand\true{\mathit{true}} \newcommand\false{\mathit{false}}\newcommand\sNull{\mathsf{simplifyNull}}$$

Here are three simple rules. They state that empty string regex accepts the empty string, $\trm{Null}$ rejects the empty string and a regex of the form $\trm{Star}(t)$ accepts the empty string as long as $t \not= Null$. Additionally, there are two rules for $\trm{Atom}$, as well.

$$\semRule{}{\acc(\trm{EmptyStr}) = \true}{empty-str}$$
$$\semRule{}{\acc(\trm{Null}) = \false}{null}$$
$$\semRule{s \not= ""}{\acc(\trm{Atom}(s)) = \false}{atom-non-empty}$$
$$\semRule{s = ""}{\acc(\trm{Atom}(s)) = \true}{atom-empty}$$
$$\semRule{t \not= Null}{\acc(\trm{Star}(t)) = \true}{kleene-star}$$

The rules for Or and Seq.
$$\semRule{ \acc(t_1) \ \textbf{or}\ \acc(t_2) }{\acc(Or(t_1, t_2)) = \true }{or-rule}$$
$$\semRule{ \acc(t_1) \ \textbf{and}\ \acc(t_2) }{\acc(Seq(t_1, t_2)) = \true }{or-rule}$$

We will skip the remaining rules for $\trm{Or}(...)$, $\trm{Seq}(...)$ and $\trm{Star}(\trm{Null})$ for which the function evaluates to  $\false$ but you should be able to figure those out yourself.

Run the definitions for Regex below and implement the function `acceptsEmpty(r: Regex): Boolean` as specified by the rules above.


In [2]:
import scala.language.postfixOps
/* Please ensure that you run this cell */
sealed trait Regex { 
    // The operators below and the "implicit def" at the bottom 
    // are for convenience. For e.g., we can write "world".star 
    // to mean Star(Atom("world")) 
    def | (r2: Regex) = Or(this, r2) // or
    def o (r2: Regex) = Seq(this, r2) // semicolon operator
    def star = Star(this) // Kleene star
}
case object EmptyStr extends Regex
case object Null extends Regex
case class Atom(s: String) extends Regex
case class Or(r1: Regex, r2: Regex) extends Regex
case class Seq(r1: Regex, r2: Regex) extends Regex
case class Star(r: Regex) extends Regex

/*-- implicitly convert strings to Atoms --*/
implicit def to_regex(s:String): Regex = Atom(s)

cmd2.sc:19: implicit conversion method to_regex should be enabled
by making the implicit value scala.language.implicitConversions visible.
This can be achieved by adding the import clause 'import scala.language.implicitConversions'
or by setting the compiler option -language:implicitConversions.
See the Scaladoc for value scala.language.implicitConversions for a discussion
why the feature should be explicitly enabled.
implicit def to_regex(s:String): Regex = Atom(s)
             ^


import scala.language.postfixOps
defined trait Regex
defined object EmptyStr
defined object Null
defined class Atom
defined class Or
defined class Seq
defined class Star
defined function to_regex

In [3]:
val r1 = ("x" o ("_yellow").star).star | ("_white")

r1: Or = Or(
  r1 = Star(r = Seq(r1 = Atom(s = "x"), r2 = Star(r = Atom(s = "_yellow")))),
  r2 = Atom(s = "_white")
)

In [3]:
/* Implement the acceptsEmpty function */
// YOUR CODE HERE


In [4]:
val r1: Regex = "" // This is where the implicit function we defined above helps us
assert(acceptsEmpty(r1), "Test # 1 failed")

val r2 = "".star
assert(acceptsEmpty(r2), "Test # 2 failed")

val r3 = "hello" | "world"
assert(!acceptsEmpty(r3), "Test # 3 failed")

val r4 = "hello" | "world".star 
assert (acceptsEmpty(r4), "Test # 4 failed")

val r5: Regex = "hello"
assert(!acceptsEmpty(r5), "Test # 5 failed")

val r6: Regex = "hello" o "world".star
assert(!acceptsEmpty(r6), "Test #6 failed")

val r7: Regex = "" o "world"
assert(!acceptsEmpty(r7), "Test # 7 failed")

val r8: Regex = ("" o "world".star ) |  "csci3155"o "hello".star
assert(acceptsEmpty(r8), "Test # 8 failed")

val r9: Regex = ("" o "world".star ) |  ("csci3155"o "hello".star)
assert(acceptsEmpty(r9), "Test # 9 failed")

val r10: Regex = Null | "hello"
assert(!acceptsEmpty(r10), "Test # 10 failed")

val r11:Regex =  Null.star
assert(!acceptsEmpty(r11), "Test #11 failed")

passed(10)

cmd4.sc:2: not found: value acceptsEmpty
val res4_1 = assert(acceptsEmpty(r1), "Test # 1 failed")
                    ^
cmd4.sc:5: not found: value acceptsEmpty
val res4_3 = assert(acceptsEmpty(r2), "Test # 2 failed")
                    ^
cmd4.sc:8: not found: value acceptsEmpty
val res4_5 = assert(!acceptsEmpty(r3), "Test # 3 failed")
                     ^
cmd4.sc:11: not found: value acceptsEmpty
val res4_7 = assert (acceptsEmpty(r4), "Test # 4 failed")
                     ^
cmd4.sc:14: not found: value acceptsEmpty
val res4_9 = assert(!acceptsEmpty(r5), "Test # 5 failed")
                     ^
cmd4.sc:17: not found: value acceptsEmpty
val res4_11 = assert(!acceptsEmpty(r6), "Test #6 failed")
                      ^
cmd4.sc:20: not found: value acceptsEmpty
val res4_13 = assert(!acceptsEmpty(r7), "Test # 7 failed")
                      ^
cmd4.sc:23: not found: value acceptsEmpty
val res4_15 = assert(acceptsEmpty(r8), "Test # 8 failed")
                     ^
cmd4.sc:26: not foun

## Simplify Null 

Next we specify some rules for getting rid of $\trm{Null}$. These rules are deliberately setup to avoid doing expensive "full" simplifications. Please read them carefully and implement  them as specified below.

If a term is an atom or null or an empty string, simplifying the term just yields the original term  back.
$$\semRule{t \in \{  \trm{Atom}(s), \trm{Null}, \trm{EmptyStr} \} }{\sNull(t) = t }{simplify-base}$$

Here are the rules for $\trm{Or}$: if one of the two arguments simplifies to $\trm{Null}$, we remove $\trm{Or}$ and replace it by the simplified version of the other term.

$$\semRule{\sNull(t_1) = \trm{Null} }{\sNull( \trm{Or}(t_1, t_2) ) = \sNull(t_2) }{simplify-or-1}$$

$$\semRule{\sNull(t_1) = s_1, s_1 \not= \trm{Null}, \text{and}\ \sNull(t_2) = \trm{Null} }{\sNull( \trm{Or}(t_1, t_2) ) = s_1 }{simplify-or-2}$$

$$\semRule{\sNull(t_1) = s_1,\ s_1 \not= \trm{Null},\ \sNull(t_2) = s_2,\ \text{and}\ s_2 \not= \trm{Null} }{\sNull( \trm{Or}(t_1, t_2) ) = \trm{Or}(s_1, s_2) }{simplify-or-2}$$

Here are the rules for $\trm{Seq}$:


$$\semRule{\sNull(t_1) = \trm{Null} }{\sNull( \trm{Seq}(t_1, t_2) ) = \trm{Null} }{simplify-seq-1}$$

$$\semRule{\sNull(t_1)= s_1\ \text{and}\ s_1 \not= \trm{Null}}{\sNull( \trm{Seq}(t_1, t_2) ) = \trm{Seq}(s_1, t_2) }{simplify-seq-2}$$

Notice that we do not bother simplifying $t_2$ for the $\trm{Seq}$ operator. We simplify $t_1$ and the two rules do different things based on whether $t_1$ simplifies to $\trm{Null}$ or not.

Here are the rules for $\trm{Star}$:
$$\semRule{\sNull(t_1) = \trm{Null} }{\sNull( \trm{Star}(t_1) ) = \trm{Null} }{simplify-star-1}$$
$$\semRule{\sNull(t_1)= s_1\ \text{and}\ s_1 \not= \trm{Null}}{\sNull( \trm{Star}(t_1) ) = \trm{Star}(s_1) }{simplify-star-2}$$

Implement a function `simplifyNull(r: Regex): Regex` following the specifications above.

In [4]:
// YOUR CODE HERE
???

scala.NotImplementedError: an implementation is missing

In [5]:
val r1 = Atom("hello") | (Null o Atom("World"))
val e1 = Atom("hello")
testWithMessage(simplifyNull(r1), e1, "1")

val r2 = Atom("hello") | (Null.star o Atom("World"))
val e2 = Atom("hello")
testWithMessage(simplifyNull(r2), e2, "2")


val r3 = (Null.star o Atom("World")) | Atom("excellent")
val e3 = Atom("excellent")
testWithMessage(simplifyNull(r3), e3, "3")

val r4 = (Null.star o Atom("sophisticated")) | (Null.star o Null)
val e4 = Null
testWithMessage(simplifyNull(r4), e4, "4")

val r5 = Atom("star rise") | r4.star | Atom("very nice")
val e5 = Atom("star rise") | Atom("very nice")
testWithMessage(simplifyNull(r5), e5, "5")
passed(15)

cmd5.sc:3: not found: value simplifyNull
val res5_2 = testWithMessage(simplifyNull(r1), e1, "1")
                             ^
cmd5.sc:7: not found: value simplifyNull
val res5_5 = testWithMessage(simplifyNull(r2), e2, "2")
                             ^
cmd5.sc:12: not found: value simplifyNull
val res5_8 = testWithMessage(simplifyNull(r3), e3, "3")
                             ^
cmd5.sc:16: not found: value simplifyNull
val res5_11 = testWithMessage(simplifyNull(r4), e4, "4")
                              ^
cmd5.sc:20: not found: value simplifyNull
val res5_14 = testWithMessage(simplifyNull(r5), e5, "5")
                              ^
Compilation Failed

In [5]:
//Tests for Seq
val r6 = Null o "Python"
val e6 = Null
testWithMessage(simplifyNull(r6), e6, "6")

println("Remember according to semantic rules for seq, you should not simplify the second argument")
val r7 =  "Python" o Null.star
val e7 = r7
testWithMessage(simplifyNull(r7), e7, "7")

val r8 =  ("Python" | Null.star) o ( Null.star o "Scala")
val e8 = "Python" o (Null.star o "Scala")
testWithMessage(simplifyNull(r8), e8, "8")

passed(5)

cmd5.sc:4: not found: value simplifyNull
val res5_2 = testWithMessage(simplifyNull(r6), e6, "6")
                             ^
cmd5.sc:9: not found: value simplifyNull
val res5_6 = testWithMessage(simplifyNull(r7), e7, "7")
                             ^
cmd5.sc:13: not found: value simplifyNull
val res5_9 = testWithMessage(simplifyNull(r8), e8, "8")
                             ^
Compilation Failed

## Derivatives 

We will now define the "derivative" of a regular expression `r` with respect to a character `c` $\delta(r, c)$ using the semantic rules below.

$$\semRule{r \in \{  \trm{EmptyStr}, \trm{Null} \} }{\delta(r, c) = \trm{Null}}{deriv-empty-null}$$

Rules for `Atom`. For a string $s$, we will denote its first characted by $s.head$ and the substring from second position to end as $s.tail$.

$$\semRule{}{\delta(\trm{Atom}(""), c) = \trm{Null}}{deriv-empty-atom}$$
$$\semRule{s.head = c}{\delta(\trm{Atom}(s), c) = \trm{Atom}(s.tail)}{deriv-atom-1}$$
$$\semRule{s.head \not= c}{\delta(\trm{Atom}(s), c) = \trm{Null} }{deriv-atom-2}$$

Notice that if an atom starts with the character `c`, then the derivative is the tail of the contained string. But if an atom does not start with the character `c`, its derivative is simply null denoting a match failure.

Rules for `Seq`.

If the first sub-term $t_1$ is not accepting then we take the derivative of $t_1$ as specified below:

$$\semRule{\acc(t_1) = \false,\ \text{and}\  s_1 = \delta(t_1, c) }{\delta(\trm{Seq}(t_1, t_2), c)) = \trm{Seq}(s_1, t_2)}{deriv-seq-1}$$

If the first sub-term $t_1$ is accepting, then the derivative of $\trm{Seq}(t_1, t_2)$ is given by the following expression obtained by taking derivative of both subterms. 

$$\semRule{\acc(t_1) = \true,\ s_1 = \delta(t_1, c),\ \text{and}\ s_2 = \delta(t_2, c) }{\delta(\trm{Seq}(t_1, t_2), c)) = \trm{Or}( \trm{Seq}(s_1, t_2), s_2 )}{deriv-seq-2}$$

Rules of `Or`.
$$\semRule{}{\delta(\trm{Or}(t_1, t_2), c)) = \trm{Or}(\delta(t_1, c), \delta(t_2, c))}{deriv-or-1}$$

Rule for `Star`.

$$\semRule{}{\delta(\trm{Star}(t)), c) = \trm{Seq}(\delta(t, c), \trm{Star}(t))}{deriv-star-1}$$

Implement the function `delta(r: Regex, c: Char): Regex` using the semantic rules above.

In [5]:
// YOUR CODE HERE
???

scala.NotImplementedError: an implementation is missing

In [6]:
// Test for Atom
val r1 = Atom("hello")
val e1 = Atom("ello")
testWithMessage(delta(r1, 'h'), e1, "#1")

val r2 = Atom("hello")
val e2 = Null
testWithMessage(delta(r2, 'c'), e2, "#2")

// Test for EmptyStr


val r3 = EmptyStr
testWithMessage(delta(r3, 'a'), Null, "#3")

// Test for Null

val r4 = Null
testWithMessage(delta(r4, 'a'), Null, "#4")

// Atom empty str
val r5 = Atom("")
testWithMessage(delta(r5, 'x'), Null, "#5")

passed(5)

cmd6.sc:4: not found: value delta
val res6_2 = testWithMessage(delta(r1, 'h'), e1, "#1")
                             ^
cmd6.sc:8: not found: value delta
val res6_5 = testWithMessage(delta(r2, 'c'), e2, "#2")
                             ^
cmd6.sc:14: not found: value delta
val res6_7 = testWithMessage(delta(r3, 'a'), Null, "#3")
                             ^
cmd6.sc:19: not found: value delta
val res6_9 = testWithMessage(delta(r4, 'a'), Null, "#4")
                             ^
cmd6.sc:23: not found: value delta
val res6_11 = testWithMessage(delta(r5, 'x'), Null, "#5")
                              ^
Compilation Failed

In [6]:
// Test Or only
val r7 = "hello" | "world"
val e7 = Null | "orld"
testWithMessage(delta(r7,'w'), e7, "#6")
// Test Or and star
val r6 = "hello" | "world".star 
val e6 = "ello" | (Null o "world".star)
testWithMessage(delta(r6,'h'), e6, "#7")
// Test Star
val r8 = "star".star 
val e8 = "tar" o r8
testWithMessage(delta(r8, 's'), e8, "#8")
// Test Star
val r9 = "".star
testWithMessage(delta(r9, 'x'), Null o r9, "#9")
passed(5)

cmd6.sc:4: not found: value delta
val res6_2 = testWithMessage(delta(r7,'w'), e7, "#6")
                             ^
cmd6.sc:8: not found: value delta
val res6_5 = testWithMessage(delta(r6,'h'), e6, "#7")
                             ^
cmd6.sc:12: not found: value delta
val res6_8 = testWithMessage(delta(r8, 's'), e8, "#8")
                             ^
cmd6.sc:15: not found: value delta
val res6_10 = testWithMessage(delta(r9, 'x'), Null o r9, "#9")
                              ^
Compilation Failed

In [6]:
// Test Seq
val r10 = "hello" o "world"
val e10: Regex = "ello" o "world"
testWithMessage(delta(r10, 'h'), e10, "#10")
testWithMessage(delta(r10, 'k'), Null o "world", "#11")

val r11 = "" o "world"
val e11 = (Null o "world") | "orld"
testWithMessage(delta(r11, 'w'), e11, "#12")

val r12 = Star("world") o "water"
val e12 = (("orld" o Star("world")) o "water") | "ater"
testWithMessage(delta(r12,'w'), e12, "#13")
passed(5)


cmd6.sc:4: not found: value delta
val res6_2 = testWithMessage(delta(r10, 'h'), e10, "#10")
                             ^
cmd6.sc:5: not found: value delta
val res6_3 = testWithMessage(delta(r10, 'k'), Null o "world", "#11")
                             ^
cmd6.sc:9: not found: value delta
val res6_6 = testWithMessage(delta(r11, 'w'), e11, "#12")
                             ^
cmd6.sc:13: not found: value delta
val res6_9 = testWithMessage(delta(r12,'w'), e12, "#13")
                             ^
Compilation Failed

We are now ready to implement our regular expression matcher. You are given  a regex `r` and a string `s`. We wish to know if the entire string `s` matches `r`. We do the following: 

  1. Start with initial regex `r`
  2. Iterate through each character `c` of the string:
     - compute derivative $\delta(r, c)$, call the result $r'$.
     - assign $r'' = \sNull(r')$ and update $r$ to equal $r''$.
  3. If at the end of the string, $\acc(r)$ is true then we have a match, else we do not have a match.

Here is a loopy version for your reference.

In [6]:
def match_loop(r0: Regex, s: String) : Boolean = {
    var r = r0
    for (c <- s) { // Iterate through the string
        r  = simplifyNull(delta(r, c))
    }
    acceptsEmpty(r)
}

cmd6.sc:4: not found: value simplifyNull
        r  = simplifyNull(delta(r, c))
             ^
cmd6.sc:4: not found: value delta
        r  = simplifyNull(delta(r, c))
                          ^
cmd6.sc:6: not found: value acceptsEmpty
    acceptsEmpty(r)
    ^
Compilation Failed

In [6]:
val r12 = Star("world") o "water"
println("1 -> "+ match_loop(r12, "worldworldwater"))
println("2 -> "+match_loop(r12, "worldwaterwater"))
println("3 -> "+match_loop(r12, "water"))
println("4 -> "+match_loop(r12, "waterworldworld"))

val r13 = ("x" o ("_yellow").star).star | ("_white")
println("5 -> "+match_loop(r13, "x"))
println("6 -> "+match_loop(r13, "x_yellow_yellowx_yellow_yellow_yellowxxxxx"))
println("7 -> "+match_loop(r13, "_white"))

val r14 = r13.star
println("8 ->" +  match_loop(r14,"x_yellow_whitex_yellow_yellow_whitexxxx_yellow_whitex"))


cmd6.sc:2: not found: value match_loop
val res6_1 = println("1 -> "+ match_loop(r12, "worldworldwater"))
                              ^
cmd6.sc:3: not found: value match_loop
val res6_2 = println("2 -> "+match_loop(r12, "worldwaterwater"))
                             ^
cmd6.sc:4: not found: value match_loop
val res6_3 = println("3 -> "+match_loop(r12, "water"))
                             ^
cmd6.sc:5: not found: value match_loop
val res6_4 = println("4 -> "+match_loop(r12, "waterworldworld"))
                             ^
cmd6.sc:8: not found: value match_loop
val res6_6 = println("5 -> "+match_loop(r13, "x"))
                             ^
cmd6.sc:9: not found: value match_loop
val res6_7 = println("6 -> "+match_loop(r13, "x_yellow_yellowx_yellow_yellow_yellowxxxxx"))
                             ^
cmd6.sc:10: not found: value match_loop
val res6_8 = println("7 -> "+match_loop(r13, "_white"))
                             ^
cmd6.sc:13: not found: value match_loop
val res6_10 = prin

Just like lists, scala String API supports `head` and `tail` operations. Use these to write a recursive version of match below.

In [6]:

def match_rec(r: Regex, s: String) : Boolean = {
    // YOUR CODE HERE
    ???
}


defined function match_rec

In [7]:
val r12 = Star("world") o "water"
testWithMessage( match_rec(r12, "worldworldwater"), match_loop(r12, "worldworldwater"), "#1")
testWithMessage( match_rec(r12, "worldwaterwater"), match_loop(r12, "worldwaterwater"), "#2")
testWithMessage( match_rec(r12, "water"), match_loop(r12, "water"), "#3")
testWithMessage(match_rec(r12, "waterworldworld"), match_loop(r12, "waterworldworld"), "#4")

val r13 = ("x" o ("_yellow").star).star | ("_white")
testWithMessage(match_rec(r13,"x"), match_loop(r13, "x"), "#5")
testWithMessage(match_rec(r13, "x_yellow_yellowx_yellow_yellow_yellowxxxxx"),
                match_loop(r13, "x_yellow_yellowx_yellow_yellow_yellowxxxxx"),
                "#6")
testWithMessage(match_rec(r13, "_white"), match_loop(r13, "_white"), "#7")

val r14 = r13.star
testWithMessage( match_rec(r14,"x_yellow_whitex_yellow_yellow_whitexxxx_yellow_whitex"),
                match_loop(r14,"x_yellow_whitex_yellow_yellow_whitexxxx_yellow_whitex"), "#8")

passed(10)

cmd7.sc:2: not found: value match_loop
val res7_1 = testWithMessage( match_rec(r12, "worldworldwater"), match_loop(r12, "worldworldwater"), "#1")
                                                                 ^
cmd7.sc:3: not found: value match_loop
val res7_2 = testWithMessage( match_rec(r12, "worldwaterwater"), match_loop(r12, "worldwaterwater"), "#2")
                                                                 ^
cmd7.sc:4: not found: value match_loop
val res7_3 = testWithMessage( match_rec(r12, "water"), match_loop(r12, "water"), "#3")
                                                       ^
cmd7.sc:5: not found: value match_loop
val res7_4 = testWithMessage(match_rec(r12, "waterworldworld"), match_loop(r12, "waterworldworld"), "#4")
                                                                ^
cmd7.sc:8: not found: value match_loop
val res7_6 = testWithMessage(match_rec(r13,"x"), match_loop(r13, "x"), "#5")
                                                 ^
cmd7.sc:10: n

Congratulations! You have now implemented a decent regex matcher using functional programming principles. We will eliminate the use of recursion later in this course using continuations and trampolines. 

## That's All Folks!